In [ ]:
import torch
import pandas as pd
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
import copy
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

In [ ]:
mnist_df = pd.read_csv('mnist_train.csv')
mnist_df

In [ ]:
mnist_test_df = pd.read_csv('mnist_test.csv')
mnist_test_df

Train a base model on the whole dataset

In [ ]:
mnist_X = mnist_df.drop(columns=["label"]).values
mnist_y = mnist_df['label'].values

In [ ]:
mnist_test_X = mnist_test_df.drop(columns=["label"]).values
mnist_test_y = mnist_test_df['label'].values

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28, 28).astype("float32") / 255.0
        img = torch.tensor(img).unsqueeze(0)
        img = img.repeat(3, 1, 1)

        img = F.interpolate(img.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        label = torch.tensor(self.y[idx], dtype=torch.long)
        return img, label
    

train_dataset = MNISTDataset(mnist_X, mnist_y)
test_dataset = MNISTDataset(mnist_test_X, mnist_test_y)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
base_model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
base_model.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)

In [ ]:
base_model.to(device)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr=0.001)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

base_loss_list = []
base_accuracy_list = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    base_model.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = base_model(X_batch)
        loss = loss_fn(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()


    avg_train_loss = epoch_loss / len(train_loader)
    base_loss_list.append(avg_train_loss)


    base_model.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = base_model(X_batch)
            test_loss = loss_fn(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    base_accuracy_list.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(base_model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")   

In [ ]:
with torch.no_grad():
    base_model.load_state_dict(best_model_weights)
    base_model.eval()
    base_model.to(device)

    y_pred = []
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = base_model(X_batch)
        preds = torch.argmax(outputs, dim=1)
        y_pred.extend(preds.cpu().numpy())

accuracy = accuracy_score(mnist_test_y, y_pred)
conf_matrix = confusion_matrix(mnist_test_y, y_pred)
class_report = classification_report(mnist_test_y, y_pred)

In [ ]:
print(f"Accuracy of base model = {accuracy}")
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Base Model')
plt.show()
print(class_report)

Dividing training data into 10% labelled and 90% unlabelled

In [ ]:
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

for _, subset_idx in strat_split.split(mnist_X, mnist_y):
    labeled_X = mnist_X[subset_idx]
    y = mnist_y[subset_idx]
    unlabeled_X = mnist_X[_]

In [ ]:
strat_split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for test, train in strat_split1.split(labeled_X, y):
    X_train = labeled_X[train]
    y_train = y[train]
    X_val = labeled_X[test]
    y_val = y[test]

In [ ]:
labeled_train_dataset = MNISTDataset(labeled_X, y)
train_loader = DataLoader(labeled_train_dataset, batch_size=64, shuffle=True)

val_dataset = MNISTDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

unlabeled_dataset = MNISTDataset(unlabeled_X, np.zeros(len(unlabeled_X)))
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

In [ ]:
model1 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model1.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)

model1.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

loss_list_10 = []
accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model1.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model1(X_batch)
        loss = loss_fn(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()


    avg_train_loss = epoch_loss / len(train_loader)
    loss_list_10.append(avg_train_loss)


    model1.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model1(X_batch)
            test_loss = loss_fn(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model1.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")   

In [ ]:
pseudo_images_07 = []
pseudo_labels_07 = []
pseudo_images_08 = []
pseudo_labels_08 = []
pseudo_images_09 = []
pseudo_labels_09 = []

with torch.no_grad():
    model1.load_state_dict(best_model_weights)
    model1.eval()
    model1.to(device)

    for images, _ in unlabeled_X:
        images = images.to(device)
        outputs = model1(images)
        preds = F.softmax(outputs, dim=1)

        conf, preds = torch.max(preds, dim=1)
        mask_07 = conf >= 0.7
        mask_08 = conf >= 0.8
        mask_09 = conf >= 0.9

        pseudo_images_07.append(images[mask_07].cpu())
        pseudo_labels_07.append(preds[mask_07].cpu())
        pseudo_images_08.append(images[mask_08].cpu())
        pseudo_labels_08.append(preds[mask_08].cpu())
        pseudo_images_09.append(images[mask_09].cpu())
        pseudo_labels_09.append(preds[mask_09].cpu())

pseudo_images_07 = torch.cat(pseudo_images_07, dim=0)
pseudo_labels_07 = torch.cat(pseudo_labels_07, dim=0)
pseudo_images_08 = torch.cat(pseudo_images_08, dim=0)
pseudo_labels_08 = torch.cat(pseudo_labels_08, dim=0)
pseudo_images_09 = torch.cat(pseudo_images_09, dim=0)
pseudo_labels_09 = torch.cat(pseudo_labels_09, dim=0)
    

In [ ]:
class PseudoDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28, 28).astype("float32") / 255.0
        img = torch.tensor(img).unsqueeze(0)
        img = img.repeat(3, 1, 1)

        img = F.interpolate(img.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        label = torch.tensor(self.y[idx], dtype=torch.long)
        return img, label
    
combined_dataset_07 = ConcatDataset([PseudoDataset(pseudo_images_07, pseudo_labels_07), labeled_train_dataset])
combined_loader_07 = DataLoader(combined_dataset_07, batch_size=64, shuffle=True)

combined_dataset_08 = ConcatDataset([PseudoDataset(pseudo_images_08, pseudo_labels_08), labeled_train_dataset])
combined_loader_08 = DataLoader(combined_dataset_08, batch_size=64, shuffle=True)

combined_dataset_09 = ConcatDataset([PseudoDataset(pseudo_images_09, pseudo_labels_09), labeled_train_dataset])
combined_loader_09 = DataLoader(combined_dataset_09, batch_size=64, shuffle=True)

In [ ]:
model1_07 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model1_07.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model1_07.to(device)
loss_fn_07 = torch.nn.CrossEntropyLoss()
optimizer_07 = torch.optim.Adam(model1_07.parameters(), lr=0.001)

model1_08 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model1_08.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model1_08.to(device)
loss_fn_08 = torch.nn.CrossEntropyLoss()
optimizer_08 = torch.optim.Adam(model1_08.parameters(), lr=0.001)

model1_09 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model1_09.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model1_09.to(device)
loss_fn_09 = torch.nn.CrossEntropyLoss()
optimizer_09 = torch.optim.Adam(model1_09.parameters(), lr=0.001)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model1_07.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_07:
        optimizer_07.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model1_07(X_batch)
        loss = loss_fn_07(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_07.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model1_07.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model1_07(X_batch)
            test_loss = loss_fn_07(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model1_07.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model1_07.load_state_dict(best_model_weights)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model1_08.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_08:
        optimizer_08.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model1_08(X_batch)
        loss = loss_fn_08(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_08.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model1_08.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model1_08(X_batch)
            test_loss = loss_fn_08(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model1_08.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model1_08.load_state_dict(best_model_weights)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model1_09.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_09:
        optimizer_09.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model1_09(X_batch)
        loss = loss_fn_09(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_09.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model1_09.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model1_09(X_batch)
            test_loss = loss_fn_09(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model1_09.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model1_09.load_state_dict(best_model_weights)

In [ ]:
with torch.no_grad():
    model1_07.eval()
    model1_07.to(device)
    model1_08.eval()
    model1_08.to(device)
    model1_09.eval()
    model1_09.to(device)

    y_pred = []
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs_07 = model1_07(X_batch)
        outputs_08 = model1_08(X_batch)
        outputs_09 = model1_09(X_batch)

        preds_07 = torch.argmax(outputs_07, dim=1)
        preds_08 = torch.argmax(outputs_08, dim=1)
        preds_09 = torch.argmax(outputs_09, dim=1)


accuracy_07 = accuracy_score(mnist_test_y, preds_07)
conf_matrix_07 = confusion_matrix(mnist_test_y, preds_07)
class_report_07 = classification_report(mnist_test_y, preds_07)

accuracy_08 = accuracy_score(mnist_test_y, preds_08)
conf_matrix_08 = confusion_matrix(mnist_test_y, preds_08)
class_report_08 = classification_report(mnist_test_y, preds_08)

accuracy_09 = accuracy_score(mnist_test_y, preds_09)
conf_matrix_09 = confusion_matrix(mnist_test_y, preds_09)
class_report_09 = classification_report(mnist_test_y, preds_09)

In [ ]:
print(f"Accuracy of model trained with 10% labeled data and pseudo-labeling:")
print(f"0.7 Confidence Threshold: {accuracy_07}")
print(f"0.8 Confidence Threshold: {accuracy_08}")
print(f"0.9 Confidence Threshold: {accuracy_09}")

plt.figure(figsize=(30, 8))
for i in range(3):
    plt.subplot(1, 3, i+1)
    if i == 0:
        sns.heatmap(conf_matrix_07, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.7 Threshold)')
    elif i == 1:
        sns.heatmap(conf_matrix_08, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.8 Threshold)')
    else:
        sns.heatmap(conf_matrix_09, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.9 Threshold)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
plt.show()

print("Classification Report (0.7 Threshold):")
print(class_report_07)
print("\nClassification Report (0.8 Threshold):")
print(class_report_08)
print("\nClassification Report (0.9 Threshold):")
print(class_report_09)

Training a model on 20% labeled and 80% unlabeled data

In [ ]:
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for _, subset_idx in strat_split.split(mnist_X, mnist_y):
    labeled_X = mnist_X[subset_idx]
    y = mnist_y[subset_idx]
    unlabeled_X = mnist_X[_]

In [ ]:
strat_split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for test, train in strat_split1.split(labeled_X, y):
    X_train = labeled_X[train]
    y_train = y[train]
    X_val = labeled_X[test]
    y_val = y[test]

In [ ]:
labeled_train_dataset = MNISTDataset(labeled_X, y)
train_loader = DataLoader(labeled_train_dataset, batch_size=64, shuffle=True)

val_dataset = MNISTDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

unlabeled_dataset = MNISTDataset(unlabeled_X, np.zeros(len(unlabeled_X)))
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

In [ ]:
model2 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model2.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)

model2.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

loss_list_10 = []
accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model2.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model2(X_batch)
        loss = loss_fn(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()


    avg_train_loss = epoch_loss / len(train_loader)
    loss_list_10.append(avg_train_loss)


    model2.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model2(X_batch)
            test_loss = loss_fn(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model2.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")   

In [ ]:
pseudo_images_07 = []
pseudo_labels_07 = []
pseudo_images_08 = []
pseudo_labels_08 = []
pseudo_images_09 = []
pseudo_labels_09 = []

with torch.no_grad():
    model2.load_state_dict(best_model_weights)
    model2.eval()
    model2.to(device)

    for images, _ in unlabeled_X:
        images = images.to(device)
        outputs = model2(images)
        preds = F.softmax(outputs, dim=1)

        conf, preds = torch.max(preds, dim=1)
        mask_07 = conf >= 0.7
        mask_08 = conf >= 0.8
        mask_09 = conf >= 0.9

        pseudo_images_07.append(images[mask_07].cpu())
        pseudo_labels_07.append(preds[mask_07].cpu())
        pseudo_images_08.append(images[mask_08].cpu())
        pseudo_labels_08.append(preds[mask_08].cpu())
        pseudo_images_09.append(images[mask_09].cpu())
        pseudo_labels_09.append(preds[mask_09].cpu())

pseudo_images_07 = torch.cat(pseudo_images_07, dim=0)
pseudo_labels_07 = torch.cat(pseudo_labels_07, dim=0)
pseudo_images_08 = torch.cat(pseudo_images_08, dim=0)
pseudo_labels_08 = torch.cat(pseudo_labels_08, dim=0)
pseudo_images_09 = torch.cat(pseudo_images_09, dim=0)
pseudo_labels_09 = torch.cat(pseudo_labels_09, dim=0)
    

In [ ]:
class PseudoDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28, 28).astype("float32") / 255.0
        img = torch.tensor(img).unsqueeze(0)
        img = img.repeat(3, 1, 1)

        img = F.interpolate(img.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        label = torch.tensor(self.y[idx], dtype=torch.long)
        return img, label
    
combined_dataset_07 = ConcatDataset([PseudoDataset(pseudo_images_07, pseudo_labels_07), labeled_train_dataset])
combined_loader_07 = DataLoader(combined_dataset_07, batch_size=64, shuffle=True)

combined_dataset_08 = ConcatDataset([PseudoDataset(pseudo_images_08, pseudo_labels_08), labeled_train_dataset])
combined_loader_08 = DataLoader(combined_dataset_08, batch_size=64, shuffle=True)

combined_dataset_09 = ConcatDataset([PseudoDataset(pseudo_images_09, pseudo_labels_09), labeled_train_dataset])
combined_loader_09 = DataLoader(combined_dataset_09, batch_size=64, shuffle=True)

In [ ]:
model2_07 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model2_07.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model2_07.to(device)
loss_fn_07 = torch.nn.CrossEntropyLoss()
optimizer_07 = torch.optim.Adam(model2_07.parameters(), lr=0.001)

model2_08 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model2_08.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model2_08.to(device)
loss_fn_08 = torch.nn.CrossEntropyLoss()
optimizer_08 = torch.optim.Adam(model2_08.parameters(), lr=0.001)

model2_09 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model2_09.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model2_09.to(device)
loss_fn_09 = torch.nn.CrossEntropyLoss()
optimizer_09 = torch.optim.Adam(model2_09.parameters(), lr=0.001)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model2_07.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_07:
        optimizer_07.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model2_07(X_batch)
        loss = loss_fn_07(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_07.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model2_07.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model2_07(X_batch)
            test_loss = loss_fn_07(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model2_07.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model2_07.load_state_dict(best_model_weights)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model2_08.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_08:
        optimizer_08.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model2_08(X_batch)
        loss = loss_fn_08(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_08.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model2_08.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model2_08(X_batch)
            test_loss = loss_fn_08(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model2_08.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model2_08.load_state_dict(best_model_weights)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model2_09.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_09:
        optimizer_09.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model2_09(X_batch)
        loss = loss_fn_09(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_09.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model2_09.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model2_09(X_batch)
            test_loss = loss_fn_09(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model2_09.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model2_09.load_state_dict(best_model_weights)

In [ ]:
with torch.no_grad():
    model2_07.eval()
    model2_07.to(device)
    model2_08.eval()
    model2_08.to(device)
    model2_09.eval()
    model2_09.to(device)

    y_pred = []
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs_07 = model2_07(X_batch)
        outputs_08 = model2_08(X_batch)
        outputs_09 = model2_09(X_batch)

        preds_07 = torch.argmax(outputs_07, dim=1)
        preds_08 = torch.argmax(outputs_08, dim=1)
        preds_09 = torch.argmax(outputs_09, dim=1)


accuracy_07 = accuracy_score(mnist_test_y, preds_07)
conf_matrix_07 = confusion_matrix(mnist_test_y, preds_07)
class_report_07 = classification_report(mnist_test_y, preds_07)

accuracy_08 = accuracy_score(mnist_test_y, preds_08)
conf_matrix_08 = confusion_matrix(mnist_test_y, preds_08)
class_report_08 = classification_report(mnist_test_y, preds_08)

accuracy_09 = accuracy_score(mnist_test_y, preds_09)
conf_matrix_09 = confusion_matrix(mnist_test_y, preds_09)
class_report_09 = classification_report(mnist_test_y, preds_09)

In [ ]:
print(f"Accuracy of model trained with 20% labeled data and pseudo-labeling:")
print(f"0.7 Confidence Threshold: {accuracy_07}")
print(f"0.8 Confidence Threshold: {accuracy_08}")
print(f"0.9 Confidence Threshold: {accuracy_09}")

plt.figure(figsize=(30, 8))
for i in range(3):
    plt.subplot(1, 3, i+1)
    if i == 0:
        sns.heatmap(conf_matrix_07, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.7 Threshold)')
    elif i == 1:
        sns.heatmap(conf_matrix_08, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.8 Threshold)')
    else:
        sns.heatmap(conf_matrix_09, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.9 Threshold)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
plt.show()

print("Classification Report (0.7 Threshold):")
print(class_report_07)
print("\nClassification Report (0.8 Threshold):")
print(class_report_08)
print("\nClassification Report (0.9 Threshold):")
print(class_report_09)

Training a model on 30% labeled and 70% unlabeled data

In [ ]:
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for _, subset_idx in strat_split.split(mnist_X, mnist_y):
    labeled_X = mnist_X[subset_idx]
    y = mnist_y[subset_idx]
    unlabeled_X = mnist_X[_]

In [ ]:
strat_split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for test, train in strat_split1.split(labeled_X, y):
    X_train = labeled_X[train]
    y_train = y[train]
    X_val = labeled_X[test]
    y_val = y[test]

In [ ]:
labeled_train_dataset = MNISTDataset(labeled_X, y)
train_loader = DataLoader(labeled_train_dataset, batch_size=64, shuffle=True)

val_dataset = MNISTDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

unlabeled_dataset = MNISTDataset(unlabeled_X, np.zeros(len(unlabeled_X)))
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

In [ ]:
model3 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model3.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)

model3.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr=0.001)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

loss_list_10 = []
accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model3.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model3(X_batch)
        loss = loss_fn(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()


    avg_train_loss = epoch_loss / len(train_loader)
    loss_list_10.append(avg_train_loss)


    model3.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model3(X_batch)
            test_loss = loss_fn(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model3.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")   

In [ ]:
pseudo_images_07 = []
pseudo_labels_07 = []
pseudo_images_08 = []
pseudo_labels_08 = []
pseudo_images_09 = []
pseudo_labels_09 = []

with torch.no_grad():
    model3.load_state_dict(best_model_weights)
    model3.eval()
    model3.to(device)

    for images, _ in unlabeled_X:
        images = images.to(device)
        outputs = model3(images)
        preds = F.softmax(outputs, dim=1)

        conf, preds = torch.max(preds, dim=1)
        mask_07 = conf >= 0.7
        mask_08 = conf >= 0.8
        mask_09 = conf >= 0.9

        pseudo_images_07.append(images[mask_07].cpu())
        pseudo_labels_07.append(preds[mask_07].cpu())
        pseudo_images_08.append(images[mask_08].cpu())
        pseudo_labels_08.append(preds[mask_08].cpu())
        pseudo_images_09.append(images[mask_09].cpu())
        pseudo_labels_09.append(preds[mask_09].cpu())

pseudo_images_07 = torch.cat(pseudo_images_07, dim=0)
pseudo_labels_07 = torch.cat(pseudo_labels_07, dim=0)
pseudo_images_08 = torch.cat(pseudo_images_08, dim=0)
pseudo_labels_08 = torch.cat(pseudo_labels_08, dim=0)
pseudo_images_09 = torch.cat(pseudo_images_09, dim=0)
pseudo_labels_09 = torch.cat(pseudo_labels_09, dim=0)
    

In [ ]:
class PseudoDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = self.X[idx].reshape(28, 28).astype("float32") / 255.0
        img = torch.tensor(img).unsqueeze(0)
        img = img.repeat(3, 1, 1)

        img = F.interpolate(img.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        label = torch.tensor(self.y[idx], dtype=torch.long)
        return img, label
    
combined_dataset_07 = ConcatDataset([PseudoDataset(pseudo_images_07, pseudo_labels_07), labeled_train_dataset])
combined_loader_07 = DataLoader(combined_dataset_07, batch_size=64, shuffle=True)

combined_dataset_08 = ConcatDataset([PseudoDataset(pseudo_images_08, pseudo_labels_08), labeled_train_dataset])
combined_loader_08 = DataLoader(combined_dataset_08, batch_size=64, shuffle=True)

combined_dataset_09 = ConcatDataset([PseudoDataset(pseudo_images_09, pseudo_labels_09), labeled_train_dataset])
combined_loader_09 = DataLoader(combined_dataset_09, batch_size=64, shuffle=True)

In [ ]:
model3_07 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model3_07.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model3_07.to(device)
loss_fn_07 = torch.nn.CrossEntropyLoss()
optimizer_07 = torch.optim.Adam(model3_07.parameters(), lr=0.001)

model3_08 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model3_08.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model3_08.to(device)
loss_fn_08 = torch.nn.CrossEntropyLoss()
optimizer_08 = torch.optim.Adam(model3_08.parameters(), lr=0.001)

model3_09 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
model3_09.classifier[3] = torch.nn.Linear(in_features=1024, out_features=10)
model3_09.to(device)
loss_fn_09 = torch.nn.CrossEntropyLoss()
optimizer_09 = torch.optim.Adam(model3_09.parameters(), lr=0.001)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model3_07.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_07:
        optimizer_07.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model3_07(X_batch)
        loss = loss_fn_07(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_07.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model3_07.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model3_07(X_batch)
            test_loss = loss_fn_07(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model3_07.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model3_07.load_state_dict(best_model_weights)


In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model3_08.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_08:
        optimizer_08.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model3_08(X_batch)
        loss = loss_fn_08(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_08.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model3_08.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model3_08(X_batch)
            test_loss = loss_fn_08(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model3_08.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model3_08.load_state_dict(best_model_weights)

In [ ]:
epochs = 1000
best_model_loss = float('inf')
best_model_weights = None
patience = 7
patience_counter = 0

comb_loss_list_10 = []
comb_accuracy_list_10 = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model3_09.train()
    epoch_loss = 0.0

    batch = 0
    for X_batch, y_batch in combined_loader_09:
        optimizer_09.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model3_09(X_batch)
        loss = loss_fn_09(outputs, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        optimizer_09.step()


    avg_train_loss = epoch_loss / len(train_loader)
    comb_loss_list_10.append(avg_train_loss)


    model3_09.eval()
    epoch_val_loss = 0.0
    epoch_val_accuracy = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model3_09(X_batch)
            test_loss = loss_fn_09(outputs, y_batch)
            epoch_val_loss += test_loss.item()
            
            y_pred = torch.argmax(outputs, dim=1)
            accuracy = (y_pred == y_batch).float().mean()
            epoch_val_accuracy += accuracy.item()

    avg_val_loss = epoch_val_loss / len(test_loader)
    avg_val_accuracy = epoch_val_accuracy / len(test_loader)
    comb_accuracy_list_10.append(avg_val_accuracy)

    if avg_val_loss < best_model_loss:
        best_model_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model3_09.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
model3_09.load_state_dict(best_model_weights)

In [ ]:
with torch.no_grad():
    model3_07.eval()
    model3_07.to(device)
    model3_08.eval()
    model3_08.to(device)
    model3_09.eval()
    model3_09.to(device)

    y_pred = []
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs_07 = model3_07(X_batch)
        outputs_08 = model3_08(X_batch)
        outputs_09 = model3_09(X_batch)

        preds_07 = torch.argmax(outputs_07, dim=1)
        preds_08 = torch.argmax(outputs_08, dim=1)
        preds_09 = torch.argmax(outputs_09, dim=1)


accuracy_07 = accuracy_score(mnist_test_y, preds_07)
conf_matrix_07 = confusion_matrix(mnist_test_y, preds_07)
class_report_07 = classification_report(mnist_test_y, preds_07)

accuracy_08 = accuracy_score(mnist_test_y, preds_08)
conf_matrix_08 = confusion_matrix(mnist_test_y, preds_08)
class_report_08 = classification_report(mnist_test_y, preds_08)

accuracy_09 = accuracy_score(mnist_test_y, preds_09)
conf_matrix_09 = confusion_matrix(mnist_test_y, preds_09)
class_report_09 = classification_report(mnist_test_y, preds_09)

In [ ]:
print(f"Accuracy of model trained with 30% labeled data and pseudo-labeling:")
print(f"0.7 Confidence Threshold: {accuracy_07}")
print(f"0.8 Confidence Threshold: {accuracy_08}")
print(f"0.9 Confidence Threshold: {accuracy_09}")

plt.figure(figsize=(30, 8))
for i in range(3):
    plt.subplot(1, 3, i+1)
    if i == 0:
        sns.heatmap(conf_matrix_07, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.7 Threshold)')
    elif i == 1:
        sns.heatmap(conf_matrix_08, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.8 Threshold)')
    else:
        sns.heatmap(conf_matrix_09, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix (0.9 Threshold)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
plt.show()

print("Classification Report (0.7 Threshold):")
print(class_report_07)
print("\nClassification Report (0.8 Threshold):")
print(class_report_08)
print("\nClassification Report (0.9 Threshold):")
print(class_report_09)